In [1]:
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "tx generator - setup"

In [2]:
from case_code.code_downloader import CodeDownloader, get_metadata
from case_code.code_loader import CodeLoader
from case_code.code_transformer import transform, get_transformed_stats

In [3]:
from datetime import datetime
from enum import Enum


class ActionType(Enum):
    DOWNLOAD = "download"
    TRANSFORM = "transform"


def get_time_difference_desc(dt_object: datetime) -> str:
    now = datetime.now()
    time_diff = now - dt_object

    seconds = time_diff.total_seconds()
    minutes = seconds // 60
    hours = minutes // 60
    days = time_diff.days

    if seconds < 60:
        return "a few seconds ago"
    elif minutes < 60:
        return f"{int(minutes)} minute{'s' if minutes > 1 else ''} ago"
    elif hours < 24:
        return f"{int(hours)} hour{'s' if hours > 1 else ''} ago"
    else:
        return f"{days} day{'s' if days > 1 else ''} ago"


def should_continue_action(action_type: ActionType, metadata_func) -> bool:
    process_action = True  # Default to processing the action

    try:
        metadata = metadata_func()
        dt_object = datetime.fromtimestamp(metadata["last_updated"])
        readable_date = dt_object.strftime("%Y-%m-%d %H:%M:%S")

        # Prepare the prompt message based on the action type
        prompt_message = {
            ActionType.DOWNLOAD: "Do you want to download the latest files?",
            ActionType.TRANSFORM: "Do you want to process transformation?",
        }.get(action_type, "Do you want to proceed?")

        print(f"Last updated:{readable_date} ({get_time_difference_desc(dt_object)})")
        response = input(
            f"Last updated was {readable_date} ({get_time_difference_desc(dt_object)}). {prompt_message} (y/n) "
        )
        process_action = response.lower() in {"y", "Yes"}
    except FileNotFoundError as e:
        print(
            f"No {action_type.value} metadata found. Proceeding with {action_type.value}."
        )
    except Exception as e:
        print(f"Error occurred: {str(e)}. Proceeding with {action_type.value}.")

    return process_action

## Download source code


In [4]:
shall_download = should_continue_action(ActionType.DOWNLOAD, get_metadata)

Last updated:2024-08-21 11:20:20 (13 minutes ago)


In [5]:
if shall_download:
    downloader = CodeDownloader()
    downloaded_count = downloader.download()

In [6]:
metadata = get_metadata()
print(f"Total files: {metadata['total_files']}")
print(f"Total cases: {metadata['total_cases']}")

Total files: 125
Total cases: 110


## Process source code


In [7]:
shall_transform = should_continue_action(ActionType.TRANSFORM, get_transformed_stats)

Last updated:2024-08-21 11:29:13 (4 minutes ago)


In [8]:
if shall_transform:
    loader = CodeLoader()
    transformed_stats = await transform(loader=loader)

Start processing files...


Processing files: 100%|██████████| 125/125 [09:37<00:00,  4.62s/file]


In [12]:
import pandas as pd

stats = get_transformed_stats()

minutes, seconds = divmod(stats["total_time_taken"], 60)
# Construct the output message
if minutes > 0:
    print(f"Time used: {minutes}m {seconds}s")
else:
    print(f"Time used: {seconds}s")

data = {
    "Title": ["total_cases", "total_failed_cases"],
    "Count": [stats["total_cases"], stats["total_failed_cases"]],
}
pd.DataFrame(data)

Time used: 9m 38s


,Title,Count
0,total_cases,110
1,total_failed_cases,41
